Removing Images and labels of pre

In [57]:
folder_path_data = 'train4'

In [58]:
import os
import glob

def delete_images_with_keyword(folder_path, keyword):
    # Get a list of image files in the folder
    image_files = glob.glob(os.path.join(folder_path, '*.jpg')) + glob.glob(os.path.join(folder_path, '*.png'))

    # Iterate through each image file
    for image_file in image_files:
        # Check if the keyword is in the file name
        if keyword in os.path.basename(image_file):
            # Delete the image file
            os.remove(image_file)
            print(f"Deleted {image_file}")

# Replace 'folder_path' with the path to your folder containing images
folder_path = folder_path_data+'/images'
# Replace 'pre' with your keyword
keyword = 'pre'

# Call the function to delete images with the specified keyword
delete_images_with_keyword(folder_path, keyword)

# Replace 'folder_path' with the path to your folder containing images
folder_path = folder_path_data+'/labels'
delete_images_with_keyword(folder_path, keyword)


## Subtract from all images

## Main SubData Images & Label Gen Code

In [59]:
import os, shutil
from PIL import Image
import cv2

train_folder = folder_path_data
#####
lbl_list = os.listdir(train_folder+'/labels')
#####
def subtract_binary_images(image1_path, image2_path, output_path):
    image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)
    if image1.shape != image2.shape:
        print("Error: Images must have the same dimensions.")
        return
    result = cv2.subtract(image1, image2)
    cv2.imwrite(output_path, result)
    print(f"Subtracted images and saved result to '{output_path}'")

def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
    else:
        print(f"Directory '{directory}' already exists. Skipping creation.")

def create_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Deleted existing folder: {folder_path}")
    os.makedirs(folder_path)
    print(f"Created folder: {folder_path}")

def convert_png_to_jpg(png_path, jpg_path):
    png_image = Image.open(png_path)
    if png_image.mode in ('RGBA', 'LA'):
        png_image = png_image.convert('RGB')
    png_image.save(jpg_path, format='JPEG', quality=100)

def create_empty_binary_mask(width, height):
    # Create a new black image (mode '1' for binary)
    # Create a new black image (mode 'L' for 8-bit grayscale)
    mask = Image.new('L', (width, height), color=0)
    return mask


lbl_list = os.listdir(train_folder+'/labels')

for i in range(len(lbl_list)):

    tmp_lbl = lbl_list.pop(0)

    create_folder(train_folder+'/subdata'+str(i)+'/images')
    create_folder(train_folder+'/subdata'+str(i)+'/labels')
    

    for label_name in lbl_list:
        
        lbl1_path = train_folder+'/labels/'+label_name
        lbl2_path = train_folder+'/labels/'+tmp_lbl
        output_path = train_folder+'/subdata'+str(i)+'/labels/'+'sd_'+str(i)+'_'+label_name[:-9]+'_mask.png'

        subtract_binary_images(lbl1_path, lbl2_path, output_path)

        image_png = train_folder+'/images/'+label_name
        image_jpg = train_folder+'/subdata'+str(i)+'/images/'+'sd_'+str(i)+'_'+label_name[:-9]+'_post.png'

        convert_png_to_jpg(image_png,image_jpg)

        tmp_img_png = train_folder+'/images/'+tmp_lbl
        tmp_img_jpg = train_folder+'/subdata'+str(i)+'/images/'+'sd_'+str(i)+'_'+label_name[:-9]+'_pre.png'

        convert_png_to_jpg(tmp_img_png,tmp_img_jpg)

    # tmp_img_png = train_folder+'/images/'+tmp_lbl
    # tmp_img_jpg = train_folder+'/subdata'+str(i)+'/train/'+'000.jpg'
    # convert_png_to_jpg(tmp_img_png,tmp_img_jpg)

    # binary_mask = create_empty_binary_mask(1024, 1024)
    # binary_mask.save(train_folder+'/subdata'+str(i)+'/train/'+'000_mask.png')



Deleted existing folder: train4/subdata0/images
Created folder: train4/subdata0/images
Deleted existing folder: train4/subdata0/labels
Created folder: train4/subdata0/labels
Subtracted images and saved result to 'train4/subdata0/labels/sd_0_bolari_1709_patch_1_1_png_rf_e1954174c34a6ee69ee8c7073d894e36_mask.png'
Subtracted images and saved result to 'train4/subdata0/labels/sd_0_bolari_1802_patch_1_1_png_rf_81647124bd8c565e704f14428d42e0fc_mask.png'
Subtracted images and saved result to 'train4/subdata0/labels/sd_0_bolari_1812_patch_1_1_png_rf_c3e8517182202e36f3709e7e68a12ae9_mask.png'
Subtracted images and saved result to 'train4/subdata0/labels/sd_0_bolari_1902_patch_1_1_png_rf_53ee81544706e462fa08cc228841479b_mask.png'
Subtracted images and saved result to 'train4/subdata0/labels/sd_0_bolari_1906_patch_1_1_png_rf_9fa7a2caaa9a4b2d8392ba68c11a0ea5_mask.png'
Subtracted images and saved result to 'train4/subdata0/labels/sd_0_bolari_1912_patch_1_1_png_rf_20ec8e4456d5a1b4be195142d9634e81_ma

## Combine total data into one folder

In [60]:
folder_path_data = 'train4' 
create_folder(folder_path_data+'/data')

Deleted existing folder: train4/data
Created folder: train4/data


In [41]:
names_list = os.listdir(folder_path_data)[3:]
def add_prefix_to_names(names_list, prefix):
    # Iterate over each name in the list
    for i in range(len(names_list)):
        # Concatenate the prefix to the current name
        names_list[i] = prefix + names_list[i]

# Example usage:
# names_list = all_dirs  # List of names
prefix = folder_path_data+"/"  # Prefix to add to each name

add_prefix_to_names(names_list, prefix)

In [55]:
import os
import shutil

def merge_folders_contents(source_folders, destination_folder):
    # Ensure the destination folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Iterate over each source folder
    for source_folder in source_folders:
        # Iterate over each file in the source folder
        for root, _, files in os.walk(source_folder):
            for file in files:
                # Get the full path of the file in the source folder
                source_file_path = os.path.join(root, file)
                # Get the relative path of the file with respect to the source folder
                relative_path = os.path.relpath(source_file_path, source_folder)
                # Get the full destination path for the file
                destination_file_path = os.path.join(destination_folder, relative_path)
                # Ensure the directory structure exists in the destination folder
                os.makedirs(os.path.dirname(destination_file_path), exist_ok=True)
                # Copy the file to the destination folder
                shutil.copy(source_file_path, destination_file_path)

# Example usage:
source_folders = names_list  # List of source folders
destination_folder = folder_path_data+'/data'  # Destination folder to merge contents

merge_folders_contents(source_folders, destination_folder)


## Remove Unwanted

In [56]:
import cv2
import glob

# read input
for fil in glob.glob(folder_path_data+'/data/labels/*.png'):
    name = fil.split('/')[-1]
    img = cv2.imread(fil)

    img[img>=1]=255

    # convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # use thresholding
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,20))
    output = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    # output[output==255]=1
    cv2.imwrite(folder_path_data+'/data/'+name, output)
